# Apache Spark / Toree

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}

In [2]:
val spark = SparkSession.builder().config("spark.executor.memory", "2g")
                    .config("spark.driver.memory", "4g")
                    .config("spark.jars.packages","ml.dmlc.xgboost4j_2.11:1.0.0")
                    .config("spark.jars.packages","ml.dmlc.xgboost4j-spark_2.11:1.0.0")
                    .master("spark://spark-master:7077").getOrCreate();

spark = org.apache.spark.sql.SparkSession@4d2634d1


org.apache.spark.sql.SparkSession@4d2634d1

In [10]:
%AddJar file:/opt/spark-data/libs/xgb_1/xgboost4j_2.11-1.0.0.jar
%AddJar file:/opt/spark-data/libs/xgb_1/xgboost4j-spark_2.11-1.0.0.jar

Starting download from file:/opt/spark-data/libs/xgb_1/xgboost4j_2.11-1.0.0.jar
Finished download of xgboost4j_2.11-1.0.0.jar
Starting download from file:/opt/spark-data/libs/xgb_1/xgboost4j-spark_2.11-1.0.0.jar
Finished download of xgboost4j-spark_2.11-1.0.0.jar


In [19]:
val schema = new StructType(Array(
  StructField("sepal length", DoubleType, true),
  StructField("sepal width", DoubleType, true),
  StructField("petal length", DoubleType, true),
  StructField("petal width", DoubleType, true),
  StructField("class", StringType, true)))

schema = StructType(StructField(sepal length,DoubleType,true), StructField(sepal width,DoubleType,true), StructField(petal length,DoubleType,true), StructField(petal width,DoubleType,true), StructField(class,StringType,true))


StructType(StructField(sepal length,DoubleType,true), StructField(sepal width,DoubleType,true), StructField(petal length,DoubleType,true), StructField(petal width,DoubleType,true), StructField(class,StringType,true))

In [28]:
val rawInput = spark.read
                    .option("header", "true")
                    .schema(schema).csv("/opt/spark-data/iris.csv")

rawInput = [sepal length: double, sepal width: double ... 3 more fields]


[sepal length: double, sepal width: double ... 3 more fields]

In [30]:
rawInput.show()

+------------+-----------+------------+-----------+------+
|sepal length|sepal width|petal length|petal width| class|
+------------+-----------+------------+-----------+------+
|         5.1|        3.5|         1.4|        0.2|setosa|
|         4.9|        3.0|         1.4|        0.2|setosa|
|         4.7|        3.2|         1.3|        0.2|setosa|
|         4.6|        3.1|         1.5|        0.2|setosa|
|         5.0|        3.6|         1.4|        0.2|setosa|
|         5.4|        3.9|         1.7|        0.4|setosa|
|         4.6|        3.4|         1.4|        0.3|setosa|
|         5.0|        3.4|         1.5|        0.2|setosa|
|         4.4|        2.9|         1.4|        0.2|setosa|
|         4.9|        3.1|         1.5|        0.1|setosa|
|         5.4|        3.7|         1.5|        0.2|setosa|
|         4.8|        3.4|         1.6|        0.2|setosa|
|         4.8|        3.0|         1.4|        0.1|setosa|
|         4.3|        3.0|         1.1|        0.1|setos

In [39]:
rawInput.na.drop()

lastException: Throwable = null


[sepal length: double, sepal width: double ... 3 more fields]

In [31]:
import org.apache.spark.ml.feature.StringIndexer
val stringIndexer = new StringIndexer().
  setInputCol("class").
  setOutputCol("classIndex").
  fit(rawInput)
val labelTransformed = stringIndexer.transform(rawInput).drop("class")

stringIndexer = strIdx_ebba540feecd
labelTransformed = [sepal length: double, sepal width: double ... 3 more fields]


[sepal length: double, sepal width: double ... 3 more fields]

In [32]:
import org.apache.spark.ml.feature.VectorAssembler
val vectorAssembler = new VectorAssembler().
  setInputCols(Array("sepal length", "sepal width", "petal length", "petal width")).
  setOutputCol("features")
val xgbInput = vectorAssembler.transform(labelTransformed).select("features", "classIndex")

vectorAssembler = vecAssembler_5c8e139f3bc6
xgbInput = [features: vector, classIndex: double]


[features: vector, classIndex: double]

In [33]:
import ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier
val xgbParam = Map("eta" -> 0.1f,
      "missing" -> -999,
      "objective" -> "multi:softprob",
      "num_class" -> 3,
      "num_round" -> 100,
      "num_workers" -> 2)
val xgbClassifier = new XGBoostClassifier(xgbParam).
      setFeaturesCol("features").
      setLabelCol("classIndex")

xgbParam = Map(num_workers -> 2, num_class -> 3, objective -> multi:softprob, num_round -> 100, missing -> -999, eta -> 0.1)
xgbClassifier = xgbc_335a7be2015b


xgbc_335a7be2015b

In [34]:
xgbClassifier.setMaxDepth(2)

xgbc_335a7be2015b

In [35]:
val xgbClassificationModel = xgbClassifier.fit(xgbInput)

Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=172.22.0.3, DMLC_TRACKER_PORT=9091, DMLC_NUM_WORKER=2}


Name: ml.dmlc.xgboost4j.java.XGBoostError
Message: XGBoostModel training failed
StackTrace:   at ml.dmlc.xgboost4j.scala.spark.XGBoost$.postTrackerReturnProcessing(XGBoost.scala:697)
  at ml.dmlc.xgboost4j.scala.spark.XGBoost$.trainDistributed(XGBoost.scala:572)
  at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.train(XGBoostClassifier.scala:190)
  at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.train(XGBoostClassifier.scala:40)
  at org.apache.spark.ml.Predictor.fit(Predictor.scala:118)